In [ ]:
# Подключение модулей
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
import cv2, numpy as np
import keras

In [ ]:
#загрузили данные
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Размер изображений
img_width, img_height = 32, 32
# Размер мини-выборки
batch_size = 100
# Кол-во изображений для обучения
nb_train_samples = 50000
# Кол-во изображений для теста
nb_test_samples = 10000

In [ ]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# one hot encode outputs
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
# Создание экземпляра модели сети VGG16
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточную часть сети обучать не надо
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
  if layer.name == 'block1_conv1':
    trainable = True
  layer.trainable = trainable

vgg16_net.summary()

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(vgg16_net)
# Преобразуем двумерный массив vgg16 в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
# Компилируем составную сеть   'SGD'
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)# Инициализировать оптимизатор RMSprop
epochs = 20
lrate = 0.001
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False) #Adam(lr=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, 
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = epochs, batch_size=batch_size)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))